### Read date database and mapper

In [258]:
from scripts import DateFinanceReportDB
import pandas as pd
import pickle

dateDB = DateFinanceReportDB()
dateDB.loadDatabase("../database/reportsDate/dateDB")
date = dateDB.getDict()

with open("../database/reportsDate/tickerMapper", 'rb') as f:
    ticker2dateName = pickle.load(f)


### Preparation data

In [170]:
import numpy as np
def castDate(date_list : list) -> np.datetime64:
    return np.datetime64(f"{date_list[-1]:04}-{date_list[-2]:02}-{date_list[-3]:02}")

#### Annual Reports

In [267]:
reports_path = "../database/annualReports/"
end_subset = "_Y_V.csv" # _Y_D.csv || _Y_V.csv

# Prepare list of ticker to parse
wigs_list = pd.read_csv("../database/gpwCompaniesLists/WIGs.csv")
ticker_list = wigs_list.to_numpy().flatten()
ticker_list = ticker_list[~pd.isnull(ticker_list)]

merged_data = []
for ticker in ticker_list:
    # Read report
    report = pd.read_csv(reports_path+ticker+end_subset).to_numpy().transpose()[1:]
    report_years = report[:,0].astype(int)

    publication_date = np.array(date[ticker2dateName[ticker]]["Roczny"])
    publication_years = publication_date[:,2]

    for i, report_year in enumerate(report_years):
        for j, publication_year in enumerate(publication_years):
            if report_year == publication_year:
                row = report[i,:]
                row[0] = castDate(publication_date[j])
                row = np.insert(row, 0, ticker)
                merged_data.append(row)
                break

merged_data_df = pd.DataFrame(merged_data)
report = pd.read_csv(reports_path+ticker+end_subset)
columns_names = report["Unnamed: 0"].to_list()[1:]
columns_names.insert(0, "Ticker")
columns_names.insert(1, "Data")
merged_data_df.columns = columns_names

merged_data_df.to_csv("../database/mergedData/Annual_V.csv")

In [265]:
reports_path = "../database/quarterlyReports/"
end_subset = "_Q_D.csv" # _Q_D.csv || _Q_V.csv

# Prepare list of ticker to parse
wigs_list = pd.read_csv("../database/gpwCompaniesLists/WIGs.csv")
ticker_list = wigs_list.to_numpy().flatten()
ticker_list = ticker_list[~pd.isnull(ticker_list)]

merged_data = []
for ticker in ticker_list:
    # Read report
    report = pd.read_csv(reports_path+ticker+end_subset).to_numpy().transpose()[1:]
    report_years = report[:,0]

    quarter_list = ["Q1", "Q2", "Q3", "Q4"]
    quarter_year = [[],[],[],[]]
    for j in report_years:
        for i, q in enumerate(quarter_list):
            if j[-2:] == q:
                quarter_year[i].append(int(j[:-3]))

    for k, q in enumerate(quarter_list):
        publication_date = np.array(date[ticker2dateName[ticker]][q])
        if len(publication_date) == 0:
            continue
        publication_years = publication_date[:,2]

        for i, report_year in enumerate(quarter_year[k]):
            for j, publication_year in enumerate(publication_years):
                if report_year == publication_year:
                    row = report[i,:]
                    row[0] = castDate(publication_date[j])
                    row = np.insert(row, 0, ticker)
                    merged_data.append(row)
                    break

merged_data_df = pd.DataFrame(merged_data)
report = pd.read_csv(reports_path+ticker+end_subset)
columns_names = report["Unnamed: 0"].to_list()[1:]
columns_names.insert(0, "Ticker")
columns_names.insert(1, "Data")
merged_data_df.columns = columns_names

merged_data_df.to_csv("../database/mergedData/Quarterly_D.csv")